### Latest version of this notebook : https://www.patreon.com/posts/88397937
* **Free Kaggle Notebook V22 - 8 October 2024**
### Dual T4 x2 Working Again - SDXL Base 1.0 DreamBooth Working - Now We Use SDPA instead of xFormers - Configs Updated
#### To use Dual GPU during training watch this tutorial to understand logic : https://youtu.be/-uhL2nW7Ddw
* Also according to my tests, SD 1.5 better than SDXL for Kaggle since on Kaggle, we can only do FP16 SDXL training opposed to BF16
* But With SD 1.5, we can do FP32 which means no quality loss
* SD 1.5 config loads best realism model automatically that I have found
* Please read this article carefully for learning all newest findings and configurations : https://www.patreon.com/posts/full-workflow-sd-98620163
* Please read instructions carefully
* Tutorial Video 2023-11-24 > https://youtu.be/16-b1AjvyBE
* Open https://www.kaggle.com/ and click + icon and click Create button and upload this notebook
* Verify your Kaggle Phone number from here to be able to use GPUs : https://www.kaggle.com/settings
* Support me on Patreon please : https://www.patreon.com/SECourses
* [Patreon Exclusive Resources Index](https://github.com/FurkanGozukara/Stable-Diffusion/blob/main/Patreon-Posts-Index.md)
* SECourses YouTube : https://www.youtube.com/SECourses
* All reg / class images links are shared here (for both man and woman) : https://www.patreon.com/posts/massive-4k-woman-87700469
* SDXL repo name : ```stabilityai/stable-diffusion-xl-base-1.0```
* For example to use RealVisXL V5.0 change repo name to : ```SG161222/RealVisXL_V5.0```
* Use the Patreon post attached SDXL DreamBooth Config. SDXL DreamBooth much better than LoRA.
* If you need LoRA do DreamBooth and then extract LoRA.

In [ ]:
!yes | apt-get update --allow-releaseinfo-change
!yes | apt install libgoogle-perftools-dev

%cd /kaggle/working

!git clone https://github.com/FurkanGozukara/kohya_ss

%cd /kaggle/working/kohya_ss

!git checkout sd3-flux.1

#!pip install fastapi==0.99.1

!./setup.sh -n -q

!pip install xformers==0.0.27.post2 --extra-index-url https://download.pytorch.org/whl/cu121

!pip install huggingface_hub

!pip install flask
!pip install pyngrok

# Load SD 1.5 or SDXL config and prepare your dataset and start training
# Don't forget to save changes you made on the loaded config. 
# If any error happens, start GUI again

!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/Kaggle_SDXL_DreamBooth.json -O "/kaggle/working/Kaggle_SDXL_DreamBooth_Best.json"
!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/Kaggle_SD_15_DreamBooth.json -O "/kaggle/working/Kaggle_SD_15.json"
  

**Below code will download and extract 1024x1024 regularization / classification images. Replace link and file name accordingly to your need.**

**For 768x768 just uncomment them and comment 1024x1024. I suggest 768x768 for SD 1.5 config.**

**All reg / class images links are shared here : https://www.patreon.com/posts/massive-4k-woman-87700469**

**Right click the desired resolution and copy link and replace below link**

**This one is for Man class / reg images dataset for 1024x1024 pixels. Use the exact same pixels as your training images.**

In [ ]:
%cd /kaggle/working
!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/man_5200_imgs_1024x1024.zip
#!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/man_5200_imgs_768x768.zip
%cd /kaggle/working/kohya_ss

import zipfile

zip_file_path = "/kaggle/working/man_5200_imgs_1024x1024.zip"
#zip_file_path = "/kaggle/working/man_5200_imgs_768x768.zip"
extract_path = "/kaggle/working/"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    
!rm -r /kaggle/working/man_5200_imgs_1024x1024.zip
!rm -r /kaggle/working/man_5200_imgs_768x768.zip

print("Zip file extracted successfully.")

**Below code will download and extract 1024x1024 regularization / classification images. Replace link and file name accordingly to your need.**

**For 768x768 just uncomment them and comment 1024x1024. I suggest 768x768 for SD 1.5 config.**

**All reg / class images links are shared here : https://www.patreon.com/posts/massive-4k-woman-87700469**

**Right click the desired resolution and copy link and replace below link**

**This one is for Woman class / reg images dataset for 1024x1024 pixels. Use the exact same pixels as your training images.**

In [ ]:
%cd /kaggle/working
!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/woman_5200_imgs_1024x1024.zip
#!wget https://huggingface.co/MonsterMMORPG/SECourses/resolve/main/woman_5200_imgs_768x768.zip
%cd /kaggle/working/kohya_ss

import zipfile

zip_file_path = "/kaggle/working/woman_5200_imgs_1024x1024.zip"
#zip_file_path = "/kaggle/working/woman_5200_imgs_768x768.zip"
extract_path = "/kaggle/working/"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    
!rm -r /kaggle/working/woman_5200_imgs_1024x1024.zip
!rm -r /kaggle/working/woman_5200_imgs_768x768.zip

print("Zip file extracted successfully.")

### First you need to register and get a token. 
### I used gmail register >
### https://dashboard.ngrok.com/get-started/your-authtoken
### Skipped authenticator not necessary
### How to use NGROK on Kaggle video : https://youtu.be/iBT6rhH0Fjs
### [Click here](https://cdn-uploads.huggingface.co/production/uploads/6345bd89fe134dfd7a0dba40/eRmm7HjCcKhqsTNTJXy9E.png) to see how a token looks like you need to get

In [ ]:

## first put your ngrok token to the below and then run this code
## it will give a link like this at below : https://2fc5-34-134-226-xxx.ngrok-free.app
## open it and then run web ui and once web ui started that link will start working
## How to use NGROK on Kaggle new video : https://youtu.be/iBT6rhH0Fjs
import os
import threading

from flask import Flask
from pyngrok import ngrok, conf

conf.get_default().auth_token = "replace_with_your_ngrok_token"

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)

public_url = ngrok.connect(7860).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 7860))

app.config["BASE_URL"] = public_url

@app.route("/")
def index():
    return "Hello from Colab!"

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()


* Best SD 1.5 and SDXL configs are already downloaded into working directory when installing first cell
* Don't forget to load and use them
* I suggest SD 1.5 config for Kaggle
* Read more info here : https://www.patreon.com/posts/full-workflow-sd-98620163

In [ ]:
%cd /kaggle/working/kohya_ss
!mkdir -p /kaggle/temp/models
!bash gui.sh --do_not_share

**Below code will delete class / reg images folder to free up space. Execute it after you have prepared your training data via Kohya GUI. Don't forget to change folder path according to yours.**

In [ ]:
!rm -r /kaggle/working/man_5200_imgs_1024x1024
!rm -r /kaggle/working/woman_5200_imgs_1024x1024
!rm -r /kaggle/working/man_5200_imgs_768x768
!rm -r /kaggle/working/woman_5200_imgs_768x768

## You can move model files and download 1 by 1

In [ ]:
#below code will move into /kaggle/temp/models folder and list you models generated there
#for SDXL DreamBooth we have to save models into that folder because of limited space
%cd /kaggle/temp/models
!dir

In [ ]:
#then by using the below code copy model into the working directory and download 1 by 1
#don't forget to change My_DB_Kaggle_1300 with the model you have
!cp /kaggle/temp/models/My_DB_Kaggle-step00000520.safetensors /kaggle/working/My_DB_Kaggle-step00000520.safetensors

In [ ]:
#after downloading model delete it from working directory so that you can copy new model
!rm -r /kaggle/working/My_DB_Kaggle-step00000520.safetensors

## A better way is uploading models into Hugging Face and downloading from there

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_folder(
    folder_path=r"/kaggle/temp/models",
    repo_id="VideoTutorial/my_checkpoints",
    repo_type="model",
)

## The below code is for LoRA training

In [ ]:
!accelerate launch \
  --num_cpu_threads_per_process=2 \
  "./sdxl_train_network.py" \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --train_data_dir="/kaggle/working/results/img" \
  --reg_data_dir="/kaggle/working/results/reg" \
  --resolution="1024,1024" \
  --output_dir="/kaggle/working/results/model" \
  --logging_dir="/kaggle/working/results/log" \
  --network_alpha="1" \
  --save_model_as=safetensors \
  --network_module=networks.lora \
  --text_encoder_lr=0.0004 \
  --unet_lr=0.0004 \
  --network_dim=32 \
  --output_name="test_kaggle" \
  --lr_scheduler_num_cycles="8" \
  --no_half_vae \
  --learning_rate="0.0004" \
  --lr_scheduler="constant" \
  --train_batch_size="1" \
  --max_train_steps="5200" \
  --save_every_n_epochs="1" \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --optimizer_type="Adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --max_data_loader_n_workers="0" \
  --bucket_reso_steps=64 \
  --gradient_checkpointing \
  --full_fp16 \
  --xformers \
  --bucket_no_upscale \
  --noise_offset=0.0 \
  --lowram \
  --cache_latents \
  --cache_latents_to_disk


**This below code will zip entire output folder of your training. Don't forget to change path according to your naming. Also be sure that you have sufficient workspace to zip it. Easier to download all saved LoRA checkpoints**

In [ ]:
!tar -czf /kaggle/working/LoRAs.tar.gz -C /kaggle/working/results/model .